## AS02: Representação Textual

**Aluno: Vinícius Henrique Giovanini**  
**Matricula: 692225**  
**Professor: Wladmir Cardoso Brandão**  

In [4]:
from sklearn.datasets import fetch_20newsgroups
import pprint as pp
import re
import numpy
import json

In [5]:
newsgroup_train = fetch_20newsgroups(subset='train', shuffle=False, random_state=42)
pp.pprint(list(newsgroup_train.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [6]:
len(list(newsgroup_train.data))

11314

### Funções de Ajuda

In [7]:
def salvar_numpy_array_para_txt(nome_arquivo, array):
    numpy.savetxt(nome_arquivo, array, fmt='%s')

def ler_txt_para_numpy_array(nome_arquivo):
    lista = []

    with open(nome_arquivo, 'r') as arquivo:
        for linha in arquivo:
            lista.append(linha.strip())

    return numpy.array(lista)
  
def salvar_dict_arq(nome_arquivo,dict):
  with open(nome_arquivo, "w") as arquivo:
    json.dump(dict, arquivo)

In [8]:
def get_tokens(txt):
  tokens = re.sub("[^\w]", " ", txt).split()
  tokens = [w.lower() for w in tokens]
  return tokens

# Pre-processamento
Removendo caracteres especiais 

In [9]:
data_list = list(newsgroup_train.data)

In [10]:
def clean_txts(txt):
  tokens = re.sub("[^\w]", " ", txt).split()
  clean_txt = [w.lower() for w in tokens]
  return " ".join(clean_txt)

# Limpando o dataset inteiro removendo caracteres especiais e colocand to lower case
data_list_cleaned = []

for line in data_list:
 data_list_cleaned.append(clean_txts(line))

In [11]:
len(data_list), len(data_list_cleaned)

(11314, 11314)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X_counts = vect.fit_transform(data_list_cleaned)
vocabulario = vect.get_feature_names_out()

In [13]:
len(vocabulario)

130107

In [14]:
salvar_numpy_array_para_txt("../saidas/AS02/data_list_processada.txt", vocabulario)

# One-Hot Encoding

**OBS**: Professor rodei criei o vocabulario e rodei o metodo para gerar o one hot enconding para cada frase, porem a celula passou 83m rodando e não terminou de fazer de todas as frases, pois tem 11k de frases com cerca de 100k de palavras. Por conta disso rodei para poucas frases

In [25]:
def embed_frase(frase):
    palavras = frase.split()
    
    embedding_frase = numpy.zeros(len(vocabulario))
    
    for palavra in palavras:
        if palavra in vocabulario:
            indice = numpy.where(vocabulario == palavra)[0][0]
            embedding_frase[indice] = 1
    
    return embedding_frase


one_hot_encoding = {}

for i, line in enumerate(data_list_cleaned):
    one_hot_encoding[line] = embed_frase(line)
    
    if i == 50:
      break
    print(i)
    i += 1

0
1
2
3
4
5
6
7
8
9


In [ ]:
salvar_dict_arq("../saidas/AS02/one_hot_encoding.txt", one_hot_encoding)